In [1]:
import os
from typing import Annotated

from semantic_kernel.agents.open_ai import AzureAssistantAgent
from semantic_kernel.contents.streaming_file_reference_content import StreamingFileReferenceContent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:

async def download_file_content(agent, file_id: str):
    try:
        # Fetch the content of the file using the provided method
        response_content = await agent.client.files.content(file_id)

        # Get the current working directory of the file from which the script is via cmd
        current_directory = os.path.abspath('')

        # Define the path to save the image in the current directory
        file_path = os.path.join(
            current_directory,  # Use the current directory of the file
            f"{file_id}.png",  # You can modify this to use the actual filename with proper extension
        )

        # Save content to a file asynchronously
        with open(file_path, "wb") as file:
            file.write(response_content.content)

        print(f"File saved to: {file_path}")
    except Exception as e:
        print(f"An error occurred while downloading file {file_id}: {str(e)}")


async def download_response_image(agent, file_ids: list[str]):
    if file_ids:
        # Iterate over file_ids and download each one
        for file_id in file_ids:
            await download_file_content(agent, file_id)

In [4]:

# creating code interpreter agent
code_agent = await AzureAssistantAgent.create(
    kernel=Kernel(),
    service_id="code_agent",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    name="CodeAgent",
    instructions="""You are an AI assistant that can write code to help answer math questions
        Always format response using markdown.
    """,
    enable_code_interpreter=True
)

print("Creating thread...")
thread_id = await code_agent.create_thread()


try:
    file_ids: list[str] = []
   
    user_input = "What does Sine look like?"

    await code_agent.add_chat_message(
        thread_id=thread_id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
    )

    is_code: bool = False
    async for response in code_agent.invoke_stream(thread_id=thread_id):
        if is_code != response.metadata.get("code"):
            print()
            is_code = not is_code

        print(f"{response.content}", end="", flush=True)

        file_ids.extend([
            item.file_id for item in response.items if isinstance(item, StreamingFileReferenceContent)
        ])

    print()

    await download_response_image(code_agent, file_ids)

finally:
    print("Cleaning up resources...")
    if code_agent is not None:
        [await code_agent.delete_file(file_id) for file_id in code_agent.code_interpreter_file_ids]
        await code_agent.delete_thread(thread_id)
        await code_agent.delete()


Creating thread...

import numpy as np
import matplotlib.pyplot as plt

# Generate values for x
x = np.linspace(-2 * np.pi, 2 * np.pi, 1000)
# Calculate sine values for x
y = np.sin(x)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(x, y, label='y = sin(x)', color='blue')
plt.title('Sine Function')
plt.xlabel('x (radians)')
plt.ylabel('y')
plt.axhline(0, color='black', lw=0.5, ls='--')
plt.axvline(0, color='black', lw=0.5, ls='--')
plt.grid(True)
plt.legend()
plt.ylim(-1.5, 1.5)
plt.xlim(-2 * np.pi, 2 * np.pi)

# Show the plot
plt.show()

The
 graph
 above
 illustrates
 the
 sine
 function
 \(
 y
 =
 \
sin
(x
)
 \
).
 The
 function
 oscill
ates
 between
 -
1
 and
 
1
 with
 a
 period
 of
 \(
 
2
\
pi
 \
).
 The
 horizontal
 axis
 represents
 the
 input
 angle
 \(
 x
 \
)
 in
 radians
,
 while
 the
 vertical
 axis
 shows
 the
 output
 value
 of
 the
 sine
 function
.
File saved to: c:\Users\juhyunlee\workspace\notebooks\multiagent-sk\single_agent_with_ci\assistant-NknLMi8dT1BVzz